In [1]:
import os
import numpy as np
import emu.neuralynx_io as nlx
import pandas as pd
import datetime

In [2]:
proj_root = '/home/elijahc/.emu/'

seeg_root = os.path.join(proj_root,'pdil','pt_01','SEEG')
print(seeg_root)

/home/elijahc/.emu/pdil/pt_01/SEEG


In [3]:
import glob

In [74]:
def load_all_nev(path):
    nev_files = sorted(glob.glob(os.path.join(path,'*.nev')))
    nev1 = nlx.load_nev(nev_files[0])['events']
    cols = nev1.dtype.names
    for p in nev_files:
        f = nlx.load_nev(p)['events']
        for row in f:
            r = {k:v for k,v in zip(cols,row)}
            r['path'] = os.path.split(p)[-1]
            yield r
         
def df_from_raw(path,index=None):
    ev = pd.DataFrame.from_records(load_all_nev(path),index=index)
    ev['EventString'] = [str(v,'utf-8') for v in ev.EventString.values]
    ev['time'] = pd.to_datetime(ev.TimeStamp.values,unit='us')
    ev = ev.set_index('TimeStamp')
    return ev

def label_blockstart(df,threshold=220000):
    labels = ['block_start']
    for i in np.arange(len(df.index.values))[1:-1]:
#         triplet = df.index.values[i-1:i+1]
        diff_prev = int(np.abs(df.index.values[i]-df.index.values[i-1]))
        diff_next = int(np.abs(df.index.values[i]-df.index.values[i+1]))
#         print([diff_prev, diff_next])
        if  diff_prev <= threshold or diff_next <= threshold:
            labels.append('block_start')
            j=1
#         elif i<len(df.index.values) and  <= threshold:
#             labels.append('block_start')
        else:
            labels.append('trial{}_start'.format(int(j)))
            j+=1
    labels.append('trial_start')
    df['label'] = labels
    return df

In [75]:
events = load_all_nev(os.path.join(seeg_root,'raw'))
df = df_from_raw(os.path.join(seeg_root,'raw'))

# Extract only events labeled ttl
df = df[df.EventString=='ttl']

# Add ttl labels
df = label_blockstart(df)

In [79]:
df

,EventString,Extra,event_id,path,pkt_id,ttl,time,label
TimeStamp,,,,,,,,
1583329111041716,ttl,"[0, 0, 0, 0, 0, 0, 0, 0]",20,Events_0007.nev,0,1,2020-03-04 13:38:31.041716,block_start
1583329111258372,ttl,"[0, 0, 0, 0, 0, 0, 0, 0]",20,Events_0007.nev,0,1,2020-03-04 13:38:31.258372,block_start
1583329163090664,ttl,"[0, 0, 0, 0, 0, 0, 0, 0]",20,Events_0007.nev,0,1,2020-03-04 13:39:23.090664,trial1_start
1583329201939898,ttl,"[0, 0, 0, 0, 0, 0, 0, 0]",20,Events_0007.nev,0,1,2020-03-04 13:40:01.939898,trial2_start
1583329233089292,ttl,"[0, 0, 0, 0, 0, 0, 0, 0]",20,Events_0007.nev,0,1,2020-03-04 13:40:33.089292,trial3_start
1583329256138877,ttl,"[0, 0, 0, 0, 0, 0, 0, 0]",20,Events_0007.nev,0,1,2020-03-04 13:40:56.138877,trial4_start
1583329276021837,ttl,"[0, 0, 0, 0, 0, 0, 0, 0]",20,Events_0007.nev,0,1,2020-03-04 13:41:16.021837,trial5_start
1583329301404702,ttl,"[0, 0, 0, 0, 0, 0, 0, 0]",20,Events_0007.nev,0,1,2020-03-04 13:41:41.404702,trial6_start
1583329320137695,ttl,"[0, 0, 0, 0, 0, 0, 0, 0]",20,Events_0007.nev,0,1,2020-03-04 13:42:00.137695,trial7_start


In [ ]:
ncs_path = glob.glob(os.path.join(seeg_root,'raw','*.ncs'))[0]

In [ ]:
lfp = nlx.load_ncs(ncs_path)

In [ ]:
lfp['time'].shape

In [ ]:
lfp['data'].shape

In [ ]:
lfp_df = pd.DataFrame.from_records({'time':lfp['time'],'data':lfp['data']},index='time')

In [ ]:
lfp.keys()

In [ ]:
lfp_df.iloc[:13*4000].plot()

In [ ]:
lfp.keys()

In [ ]:
paths = load_all_nev(os.path.join(seeg_root,'raw'))
paths

In [ ]:
np.nargmax([len(nlx.load_nev(p)['records']) for p in paths])

In [ ]:
# df = pd.DataFrame.from_records(recs)
df

# events_df = pd.DataFrame.from_records(list(nev['events'][0]),columns=nev['events'].dtype.names)

In [ ]:
recs